In [1]:
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split

import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import to_categorical

C:\Users\windo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
class Dataset():
    DATA_DIR_NAME = 'data'
    TRAINING_IMG_FILENAME = 'train_image.pkl' 
    TRAINING_LABEL_FILENAME = 'train_label.pkl'
    TESTING_IMG_FILENAME = 'test_image.pkl'
    
    def __init__(self, validation_size=0.2):
        self.validation_size = validation_size
        self.build()
        
    def build(self):
        self.images = self.load_pickle(os.path.join(self.DATA_DIR_NAME, self.TRAINING_IMG_FILENAME))
        self.labels = self.load_pickle(os.path.join(self.DATA_DIR_NAME, self.TRAINING_LABEL_FILENAME))
        self.training_images, self.validation_images, self.training_labels, self.validation_labels = train_test_split(self.images, self.labels, test_size=self.validation_size)
        
        self.training_set = self.build_set(self.training_images, self.training_labels)
        self.validation_set = self.build_set(self.validation_images, self.validation_labels)
        self.test_set = self.load_pickle(os.path.join(self.DATA_DIR_NAME, self.TESTING_IMG_FILENAME))
        
    def build_set(self, images, labels):
        return [images, labels]
        
    def load_pickle(self, filename, mode='rb'):
        return np.array(pickle.load(open(filename, mode)))

In [9]:
d = Dataset()
print(d.training_set[0].shape, d.validation_set[0].shape)
print(d.training_set[1].shape, d.validation_set[1].shape)

(6400, 784) (1600, 784)
(6400,) (1600,)


In [14]:
class DatasetProvider():
    def __init__(self, augment=False, validation_size=0.2, batch_size=32):
        self.dataset = Dataset(validation_size)
        self.augment = augment
        self.batch_size = batch_size
    
    def training_set(self):
        for batch in self.generate_batch(self.dataset.training_set):
            yield batch
    
    def validation_set(self):
        for batch in self.generate_batch(self.dataset.validation_set):
            yield batch
    
    def generate_batch(self, img_label_set):
        while True:
            all_idx = np.arange(len(img_label_set[0]))
            random_idx = np.random.choice(all_idx, size=self.batch_size)
            random_img = img_label_set[0][random_idx]
            random_label = img_label_set[1][random_idx]
            yield self.preprocess_batch([random_img, random_label])
            
    def preprocess_batch(self, img_label_batch):
        return self.preprocess_images(img_label_batch[0]), self.preprocess_labels(img_label_batch[1])
    
    def preprocess_images(self, imgs):
        #TODO add augment code
        imgs = imgs/255.
        return np.reshape(imgs, (imgs.shape[0], 28, 28, 1))
    
    def preprocess_labels(self, labels):
        labels[labels == 2] = 1
        labels[labels == 3] = 2
        labels[labels == 6] = 3
        return to_categorical(labels, num_classes=4)

In [15]:
class ImageClassifierModel():
    
    def __init__(self):
        #TODO
        #initialize params and hyperparams'
        self.a = 2
        self.model = self.build()
    
    def build(self):
        #TODO
        #define model and build model
        input_layer = Input(shape=(28, 28, 1))
        x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(input_layer)
        x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(x)

        x = Dropout(0.7)(x)

        x = Flatten()(x)
        x = Dense(units=1024, activation='relu')(x)
        x = Dense(units=512, activation='relu')(x)
        output_layer = Dense(units=4, activation='softmax')(x)

        model = Model(inputs=input_layer, outputs=output_layer)
        
        adam = keras.optimizers.Adam(lr=0.0001)
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model
    

In [16]:
model = ImageClassifierModel()
model = model.model
dataset_provider = DatasetProvider()

In [17]:
model.fit_generator(dataset_provider.training_set(), steps_per_epoch=6400//32, epochs=10, validation_data=dataset_provider.validation_set(), validation_steps=1600//32)

Epoch 1/10
200/200 [==============================] - 6s 28ms/step - loss: 0.7132 - acc: 0.7014 - val_loss: 0.5743 - val_acc: 0.7731
Epoch 2/10
200/200 [==============================] - 4s 22ms/step - loss: 0.5210 - acc: 0.7922 - val_loss: 0.5030 - val_acc: 0.8006
Epoch 3/10
200/200 [==============================] - 4s 22ms/step - loss: 0.4575 - acc: 0.8183 - val_loss: 0.4994 - val_acc: 0.8013
Epoch 4/10
200/200 [==============================] - 4s 22ms/step - loss: 0.4229 - acc: 0.8269 - val_loss: 0.4633 - val_acc: 0.8287
Epoch 5/10
200/200 [==============================] - 4s 22ms/step - loss: 0.3824 - acc: 0.8491 - val_loss: 0.4053 - val_acc: 0.8444
Epoch 6/10
200/200 [==============================] - 4s 22ms/step - loss: 0.3491 - acc: 0.8688 - val_loss: 0.4191 - val_acc: 0.8319
Epoch 7/10
200/200 [==============================] - 4s 22ms/step - loss: 0.3539 - acc: 0.8609 - val_loss: 0.4063 - val_acc: 0.8313
Epoch 8/10
200/200 [==============================] - 4s 22ms/step - 